In [1]:
import sys
sys.path.append('/home/silja/fatepredict/fatepredict_experiments')

In [2]:
import zarr
import numpy as np
import networkx as nx
from skimage.measure import regionprops
from segmentation import segment_stats
from linajea_cost_test import get_merge_graph_from_array
from funlib.math import decode64

In [23]:
z = zarr.open('anno_alice_T2030_tiny.zarr','r')
gt_image = z['gt_trackimage'][:]
raw = z['Raw'][:]
fragments = z['Fragments'][:]

In [4]:
# annation label is 10000, 10001, 10004, 10005, 10007, 10010, 10013, 10016, 10017 pick them out
gt_mask = gt_image >=10000
gt_image = gt_mask * gt_image

In [5]:
gt_image.shape

(3, 15, 30, 30)

In [ ]:
# create annotation graph by gt_trackimage
time = 
for label in np.unique(gt_image):


In [6]:
# get the id list of one time frame
id, positions, _ = segment_stats(gt_image[0],0)

In [7]:
t = 0
merge_tree = z['Merge_tree/Merge/'+str(t)]
scores = z['Merge_tree/Scoring/'+str(t)]
merge_tree = get_merge_graph_from_array(merge_tree,scores)


In [8]:
# count the number of how many annotation id in merge tree
def count_anno_id(id,merge_tree):
    '''
    merge_tree: nx.Digrph merge_tree create by waterz 
    id: ndarray (n,) 1d array store all ids in GT image
    correct_id_num: int count the number of how many annotation node(id) in merge tree
    '''
    mt_id = list(merge_tree.nodes)
    correct_id_num = len(set(id)&set(mt_id))
    return correct_id_num 

correct_id_num = count_anno_id(id,merge_tree)

In [9]:
import napari
viewer = napari.Viewer()
viewer.add_image(raw[3])
viewer.add_labels(gt_image)

<Labels layer 'gt_image' at 0x7f09e4ed7700>

# get the sovler output

In [11]:
# read .gexf file which created by run_candidate_graph.ipynb
graph = nx.read_gexf('text.gexf')

In [41]:
def get_select_graph(graph):
    select_graph = nx.DiGraph()
    for u,v,data in graph.edges(data=True):
        if data['selected']:
            select_graph.add_edge(v, u, source = v, target = u)
            select_graph.add_node(u, **graph.nodes[u])
            select_graph.add_node(v, **graph.nodes[v])
    return select_graph

In [42]:
select_graph = get_select_graph(graph)

In [14]:
def pick_select_edge(graph):
    edges = []
    selected_edges = []
    for u,v,data in graph.edges(data=True):
        edges.append((u,v))
        if data['selected']:
            selected_edges.append((u,v))
    return edges, selected_edges

edges, selected_edges = pick_select_edge(graph)

In [15]:
from linajea_cost_test import get_merge_graph

decode64(int(25016140147197952),dims=5,bits=[9,12,12,12,19])

[0, 4, 25, 6, 711]

In [16]:
def selected_nodes_t(graph,t):
    nodes_t = []
    for node in graph.nodes:
        cor = decode64(int(node),dims=5,bits=[9,12,12,12,19])
        if cor[0] == t:
            nodes_t.append(node)
    return nodes_t

In [34]:
track_data = []
# creat a list for tracking node
track_id_node = []
track_id = 0
for node in select_graph.nodes:
    cor = decode64(int(node),dims=5,bits=[9,12,12,12,19])
    track_data.append([track_id,cor[0],cor[1],cor[2],cor[3]])
    track_id_node.append(node)
    track_id += 1

In [36]:
from candidate_graph import iterate_tree
def add_select_nodes_mask(z):
    time = 3
    cell = np.zeros(fragments.shape,dtype=int)
    for t in range(time-1):
        ids_pre = z['Fragment_stats/id/'+str(t)]
        merge_tree_pre = z['Merge_tree/Merge/'+str(t)]
        scores_pre = z['Merge_tree/Scoring/'+str(t)]

        ids_nex = z['Fragment_stats/id/'+str(t+1)]
        merge_tree_nex = z['Merge_tree/Merge/'+str(t+1)]
        scores_nex = z['Merge_tree/Scoring/'+str(t+1)]


        merge_tree = get_merge_graph_from_array(merge_tree,scores)
        for node in select_graph.nodes:
            if merge_tree.has_node(int(node)):
                # find the merge path
                sub_a = iterate_tree(merge_tree,int(node))
                # creata new cell mask for node
                mask = np.zeros(fragments[t].shape)
                #find id
                for a in sub_a:
                    if a in list(ids):
                        # a is the leave node id
                        # get the label of this leave node
                        label_a = list(ids).index(a)+1
                        mask[fragments[t] == label_a] = 1
                cell[t][mask==1] = track_id_node.index(node)
        return cell

In [ ]:
# Mask track cell as smae label
select_graph.

In [37]:
decode64(19140315621363712,dims=5,bits=[9,12,12,12,19])

[0, 8, 12, 2, 544]

In [38]:
cell = add_select_nodes_mask(z)

In [39]:
viewer.add_labels(cell)

<Labels layer 'cell [1]' at 0x7f0a060b7b80>

In [21]:
np.unique(cell)

array([  0, 173, 174])

In [43]:
select_graph.edges(data=True)

OutEdgeDataView([('1020578718816256', '2814981697443329', {'source': '1020578718816256', 'target': '2814981697443329'}), ('2814981697443329', '8374103897543682', {'source': '2814981697443329', 'target': '8374103897543682'}), ('1794634908964352', '2216847378224641', {'source': '1794634908964352', 'target': '2216847378224641'}), ('8584995398486528', '25966100969687553', {'source': '8584995398486528', 'target': '25966100969687553'}), ('25966100969687553', '70368752566786', {'source': '25966100969687553', 'target': '70368752566786'}), ('11892360728284160', '633361655664129', {'source': '11892360728284160', 'target': '633361655664129'}), ('633361655664129', '18436671112480770', {'source': '633361655664129', 'target': '18436671112480770'}), ('950218593931776', '8901869518722049', {'source': '950218593931776', 'target': '8901869518722049'}), ('8901869518722049', '950167060617730', {'source': '8901869518722049', 'target': '950167060617730'}), ('13616583976948736', '12736966088987137', {'source

In [ ]:
for source_id, target_id in gt_graph.edges():
        source_node = gt_graph.nodes[source_id]
        target_node = gt_graph.nodes[target_id]